# Final Tuned Models Results.

In [1]:
# load pickle file train from dataset/train.pkl and target from dataset/target.pkl

import pickle

with open('dataset/train.pkl', 'rb') as f:
    train = pickle.load(f)

with open('dataset/target.pkl', 'rb') as f:
    target = pickle.load(f)



In [3]:
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Define the number of folds for cross-validation
n_splits = 5

# Create an instance of KFold with the specified number of folds
kf = KFold(n_splits=n_splits)

rmse_scores_lightgbm = []

for train_index, test_index in kf.split(train):
    # Split the data into training and testing sets for the current fold
    X_train, X_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
    # Create an instance of lgb.Dataset for the training and testing sets
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_test = lgb.Dataset(X_test, y_test)

    # Define the parameters for the LightGBM model
    params = {
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 100,
    'learning_rate': 0.09841079471843048,
    'feature_fraction': 0.6146295376710438,
    'bagging_fraction': 0.6360723189013848,
    'bagging_freq': 7,
    'verbose': 0
    }

    # Train the LightGBM model
    model = lgb.train(params, lgb_train, num_boost_round=100)

    # Make predictions on the testing data
    y_pred = model.predict(X_test)

    # Calculate the RMSE for the current fold
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    # Print the RMSE for the current fold
    print("RMSE:", rmse)

    # Append the RMSE to the list of scores
    rmse_scores_lightgbm.append(rmse)


# Calculate the average RMSE across all folds
average_rmse = np.mean(rmse_scores_lightgbm)

# Print the average RMSE
print("Average RMSE:", average_rmse)

RMSE: 1.1915660940222301
RMSE: 1.1647653628494292
RMSE: 1.208591522100199
RMSE: 1.2674121916968033
RMSE: 1.1500745311872123
Average RMSE: 1.1964819403711748


In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np

# Define the number of folds for cross-validation
n_splits = 5

# Create an instance of KFold with the specified number of folds
kf = KFold(n_splits=n_splits)

# Initialize an empty list to store the RMSE for each fold
catboost_rmse_scores = []

# Specify the parameters for the CatBoost model
params = {
    'iterations': 100,
    'learning_rate': 0.15,
    'depth': 10,
    'loss_function': 'RMSE'
}


# Perform cross-validation
for train_index, test_index in kf.split(train):
    # Split the data
    X_train, X_test = train[train_index], train[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    # Create an instance of CatBoostRegressor
    model = CatBoostRegressor(**params)
    
    # Fit the model
    model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=100)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate RMSE and append to the list
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print("CatBoost RMSE:", rmse)
    catboost_rmse_scores.append(rmse)

# Calculate the average RMSE
average_catboost_rmse = np.mean(catboost_rmse_scores)
print("Average CatBoost RMSE:", average_catboost_rmse)


0:	learn: 1.9641591	test: 2.0062060	best: 2.0062060 (0)	total: 1.7s	remaining: 2m 48s
99:	learn: 1.2034665	test: 1.2582424	best: 1.2582424 (99)	total: 2m 29s	remaining: 0us

bestTest = 1.258242419
bestIteration = 99

CatBoost RMSE: 1.2582424187423533
0:	learn: 1.9768329	test: 1.9365990	best: 1.9365990 (0)	total: 1.59s	remaining: 2m 37s
99:	learn: 1.2050881	test: 1.2337259	best: 1.2337259 (99)	total: 2m 29s	remaining: 0us

bestTest = 1.233725865
bestIteration = 99

CatBoost RMSE: 1.233725864789475
0:	learn: 1.9671160	test: 1.9887284	best: 1.9887284 (0)	total: 1.47s	remaining: 2m 25s
99:	learn: 1.1998497	test: 1.2782789	best: 1.2782789 (99)	total: 2m 26s	remaining: 0us

bestTest = 1.278278939
bestIteration = 99

CatBoost RMSE: 1.2782789393517557
0:	learn: 1.9646954	test: 1.9952645	best: 1.9952645 (0)	total: 1.53s	remaining: 2m 31s
99:	learn: 1.1782875	test: 1.3373470	best: 1.3373470 (99)	total: 2m 31s	remaining: 0us

bestTest = 1.337346967
bestIteration = 99

CatBoost RMSE: 1.33734696720

In [ ]:
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout, Activation
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.utils import plot_model
from keras.models import load_model
import tensorflow.keras.backend as K
from graphviz import Digraph
from sklearn.model_selection import train_test_split

def NN_RMSLE(y_actual, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_actual))) 

train_xx, val_xx, train_yy, val_yy = train_test_split(train, target, test_size=0.2, random_state=42)

model = Sequential()

earlyStop= EarlyStopping(monitor='val_loss', mode='min', patience=3)

model.add(layers.Dense(288, activation='relu',input_shape=(train_xx.shape[1],)))
model.add(Dropout(0))
model.add(layers.Dense(224, activation='relu'))
model.add(Dropout(0.2))
model.add(layers.Dense(96, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1  ,activation='linear'))


model.compile(optimizer='adam', loss = NN_RMSLE)

model.fit(train_xx, train_yy, epochs = 15, batch_size = 2048, validation_data=(val_xx,val_yy),callbacks = earlyStop)
print('Neural Network Training RMSLE = ', model.evaluate(train, target, verbose=0))
model.save('tuned_3layer.keras')
model.summary()


C:\Users\dublu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
7755/7755 ━━━━━━━━━━━━━━━━━━━━ 48s 6ms/step - loss: 1.8226 - val_loss: 1.5620
Epoch 2/15
7755/7755 ━━━━━━━━━━━━━━━━━━━━ 46s 6ms/step - loss: 1.5756 - val_loss: 1.4997
Epoch 3/15
7755/7755 ━━━━━━━━━━━━━━━━━━━━ 46s 6ms/step - loss: 1.5227 - val_loss: 1.4749
Epoch 4/15
7755/7755 ━━━━━━━━━━━━━━━━━━━━ 46s 6ms/step - loss: 1.4958 - val_loss: 1.4590
Epoch 5/15
7755/7755 ━━━━━━━━━━━━━━━━━━━━ 45s 6ms/step - loss: 1.4801 - val_loss: 1.4383
Epoch 6/15
7755/7755 ━━━━━━━━━━━━━━━━━━━━ 49s 6ms/step - loss: 1.4667 - val_loss: 1.4383
Epoch 7/15
7755/7755 ━━━━━━━━━━━━━━━━━━━━ 48s 6ms/step - loss: 1.4582 - val_loss: 1.4182
Epoch 8/15
7755/7755 ━━━━━━━━━━━━━━━━━━━━ 46s 6ms/step - loss: 1.4499 - val_loss: 1.4148
Epoch 9/15
7755/7755 ━━━━━━━━━━━━━━━━━━━━ 46s 6ms/step - loss: 1.4429 - val_loss: 1.4075
Epoch 10/15
7755/7755 ━━━━━━━━━━━━━━━━━━━━ 45s 6ms/step - loss: 1.4374 - val_loss: 1.3986
Epoch 11/15
7755/7755 ━━━━━━━━━━━━━━━━━━━━ 46s 6ms/step - loss: 1.4315 - val_loss: 1.3864
Epoch 12/15
7755/77

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 288)            │         2,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 288)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 224)            │        64,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 96)             │        21,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            97 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 267,941 (1.02 MB)

 Trainable params: 89,313 (348.88 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 178,628 (697.77 KB)